# Preprocessing intracranial EEG using MNE-python

*NeuroHackademy 2023*  
[Liberty Hamilton, PhD](https://slhs.utexas.edu/research/hamilton-lab)  
Assistant Professor, Department of Speech, Language, and Hearing Sciences and  
Department of Neurology  
The University of Texas at Austin  

This notebook will show you how to preprocess intracranial EEG using MNE-python. This uses a freely available iEEG dataset on audiovisual movie watching from [Julia Berezutskaya, available on OpenNeuro.org](https://openneuro.org/datasets/ds003688/versions/1.0.7/metadata). This notebook mostly covers the basics of how to look at iEEG data, and my tutorial will discuss how to find and identify artifacts. The method of high gamma extraction is identical to that used in [Hamilton et al. 2018](https://doi.org/10.1016/j.cub.2018.04.033) and [Hamilton et al. 2021](https://doi.org/10.1016/j.cell.2021.07.019).

## Python libraries used in this tutorial

* MNE-python
* numpy
* pandas
* matplotlib

## What you will do in this tutorial

* Load an iEEG dataset in MNE-python
* Compare iEEG dataset with BIDs metadata vs. without so you know what to do if you encounter data without this info
* Plot the power spectrum of the data to check for bad channels and compare channel types
* Re-reference the data according to different reference schemes
* Compute the high gamma analytic amplitude of the signal
* Plot evoked data

In [1]:
%matplotlib notebook

import mne
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report
from mne_bids.path import get_bids_path_from_fname
from bids import BIDSLayout
from ecog_preproc_utils import transformData
import bids 

## Download BIDS iEEG dataset

Here we will download an example iEEG dataset from [Berezutskaya et al.  Open multimodal iEEG-fMRI dataset from naturalistic stimulation with a short audiovisual film](https://openneuro.org/datasets/ds003688/versions/1.0.7/metadata). For this tutorial we will use data from `sub-06`, `iemu` data only, which has been downloaded to the jupyter hub. The whole dataset is rather large (15 GB), so if you prefer to download just this session you can do that.

In [2]:
# Change the data directory below to where your data are located. 
parent_dir = '/home/jovyan/shared/ds003688/'  # This is on the jupyter hub
ieeg_dir = f'{parent_dir}/sub-{subj}/ses-{sess}/ieeg/'
channel_path = f'{ieeg_dir}/sub-{subj}_ses-{sess}_task-{task}_acq-{acq}_run-{run}_channels.tsv'
raw_path = f'{ieeg_dir}/sub-{subj}_ses-{sess}_task-{task}_acq-{acq}_run-{run}_ieeg.vhdr'

bids_path = get_bids_path_from_fname(raw_path)
base_name = os.path.basename(raw_path).split('.')[0]

## BIDS layout

We can use `pybids` to show a little bit about the files in this BIDS dataset. We won't get as much into this, but if you'd like to try this tutorial on your own you may wish to delve into this more.

In [ ]:
layout = BIDSLayout(parent_dir)

In [ ]:
layout.get_tasks()

In [ ]:
all_files = layout.get()
print("There are {} files in the layout.".format(len(all_files)))
print("\nThe first 10 files are:")
all_files[:10]

In [ ]:
print_dir_tree(parent_dir, max_depth=3)

## Let's load some iEEG data!

First, we will choose the relevant subject, session, task, acquisition, and run. Note that if you wish to change these variables, you may need to download the data yourself.

To show the capabilities of BIDS and contrast to when we don't use BIDS, we'll load the data in two ways. The data structure using BIDS will be called `raw`, the data structure without BIDS will be `raw_nobids`.

In [ ]:
# Change these variables to work for your block
subj = '06'
sess = 'iemu'
task = 'film'
acq = 'clinical'
run = 1

In [ ]:
# Read data and extract parameters from BIDS files
raw = read_raw_bids(bids_path, verbose=True)

In [ ]:
# Read the data assuming we didn't have the BIDS structure in place
raw_nobids = mne.io.read_raw_brainvision(raw_path, preload=True)

In [ ]:
# Let's load the data into memory and print some information about it. The 
# info structure contains a lot of helpful metadata about number of channels,
# sampling rate, data types, etc. It can also contain information about the
# participant and date of acquisition, however, this dataset has been anonymized.
raw.load_data()
raw.info

In [ ]:
raw.info['ch_names']

## Plot the power spectrum

Now we will plot the power spectrum of the signal to give us an idea of the signals we're getting. Bad channels (or channels that are not EEG/ECoG) will often have a very different power spectrum than the good channels. These will show up as highly outside the range of the other channels (either flat, or much higher/lower power).

In [ ]:
raw.compute_psd().plot(picks='data', exclude=[])

In [ ]:
# If we want to see other options we have for computing the power spectrum, 
# we can consult the help function
raw.compute_psd?

## Do the same without having loaded the metadata from BIDs

Here we will see the data with bad channels included, and with all the channel types marked as EEG.

In [ ]:
raw_nobids.info

In [ ]:
raw_nobids.compute_psd().plot()

In [ ]:
# Plot the data, reject bad segments. Look for times where there
# are spike wave discharges (epileptiform artifacts) or large
# movement artifacts. Be selective, look out for blocks with a 
# ton of seizure activity
raw.plot(scalings='auto', color=dict(eeg='b', ecog='b'), n_channels=64, block=True)

In [ ]:
# Plot the data, reject bad segments. Look for times where there
# are spike wave discharges (epileptiform artifacts) or large
# movement artifacts. Be selective, look out for blocks with a 
# ton of seizure activity. This is when we don't have the nice
# BIDS metadata automatically loaded in.
raw_nobids.plot(scalings='auto', color=dict(eeg='b', ecog='b'), n_channels=64, block=True)

## Referencing

Referencing or re-referencing your data should be done with some knowledge of your recording setup and what you wish to measure. You can read more about referencing [here (for EEG)](https://pressrelease.brainproducts.com/referencing/#:~:text=The%20reference%20influences%20the%20amplitude,affected%20by%20similar%20electrical%20activity.). Typically, experimenters will choose one of the following references:

1. Based on a single electrode in white matter (or relatively "quiet" electrode far away from your signals of interest. 
2. Based on the average of all electrodes or a block of electrodes (CAR or Common Average Reference). Note that the CAR is *not* a good idea if all of your electrodes are within a single functional area, as you will likely subtract out more signal than noise. 
3. Bipolar referencing, in which pairs of adjacent electrodes are subtracted to calculate more local signals. This is a bit more complicated but allows you to work with data in a single region without the drawbacks of the CAR.

In [ ]:
# Example of single reference channel - this subtracts the signal from this channel
# from all other channels (including itself), so the reference will appear as a flat
# line after this step
raw_ref = raw.copy()
raw_ref.set_eeg_reference(ref_channels = ['P01'], )
raw_ref.plot(scalings='auto', color=dict(eeg='b', ecog='b'), n_channels=64, block=True)

In [ ]:
# Example of common average reference. The common average reference subtracts the average
# signal across all *good* channels from every single channel. This is typically a good
# choice for removing noise across all channels (for example, sometimes EMG from chewing,
# some movement artifacts, electrical noise).
raw_ref_car = raw.copy()
raw_ref_car.set_eeg_reference(ref_channels = 'average')
raw_ref_car.plot(scalings='auto', color=dict(eeg='b', ecog='b'), n_channels=64, block=True)

## Bipolar reference

Bipolar referencing is a bit trickier and is not fully implemented here. You need to use knowledge of the physical locations of the electrodes to properly create the bipolar montage. For example, in the image below, we would need to use the knowledge of how the electrodes are placed in order to create the appropriate pairs for the anode and cathode.

![sub-06 electrode locations](sub-06_ses-iemu_acq-render_photo_ecog_left.jpeg)

In [ ]:
# Example of bipolar reference. This would normally be done with some manual
# intervention of the specific channel pairs. 
import re  # regex for comparing channel names

raw_ieeg = raw.copy()
raw_ieeg.pick_types(ecog=True)
ch_pairs = list(zip(raw_eeg.info['ch_names'][:-1],
                    raw_eeg.info['ch_names'][1:]))

# Make a list of channels for the anode and the cathode
anode = []
cathode = []
# Only subtract channels with the same device name (these will be
# close in space). This is still not ideal as we are probably
# subtracting electrodes that are far away from one another in 
# space (for example, electrodes 8 and 9 on the grid picture
# above should not be used for a bipolar reference)
for pair in ch_pairs:
    # get rid of the numbers in the ch_name
    ch1_dev = re.sub(r'\d+', '', pair[0]) 
    ch2_dev = re.sub(r'\d+', '', pair[1]) 
    # if these are part of the same device, consider them for 
    # anode and cathode selection
    if ch1_dev == ch2_dev:
        anode.append(pair[0])
        cathode.append(pair[1])

# Apply the bipolar reference
raw_ref_bip = mne.set_bipolar_reference(raw, anode=anode, cathode=cathode)
raw_ref_bip.plot(scalings='auto', color=dict(eeg='b', ecog='b'), n_channels=64, block=True)

# Calculate the high gamma transform of your data

Now we will take the raw, preprocessed data, and convert to high gamma analytic amplitude for further analysis. The high gamma analytic amplitude is used in many papers as a proxy for multi-unit firing (see [Ray and Maunsell, PLoS Biology 2011](https://journals.plos.org/plosbiology/article?id=10.1371/journal.pbio.1000610)).

This particular version of the high gamma transform uses the same procedure as used in [Hamilton et al. 2018](cell.com/current-biology/pdf/S0960-9822(18)30461-5.pdf) and [Hamilton et al. 2021](https://www.cell.com/cell/pdf/S0092-8674(21)00878-3.pdf). The basic idea is to take 8 bands within the 70-150 Hz range, calculate the Hilbert transform, then take the analytic amplitude of that signal and average across the 8 bands. This form of averaging results in higher SNR than one band between 70-150 Hz. 

In [ ]:
# Get the high gamma data
# Generally, do a CAR if you have widespread coverage over multiple
# areas (not just one sensory area)
# If you have limited coverage, you may choose to do no CAR or choose
# to reference to one specific channel.
hgdat = transformData(raw_ieeg, ieeg_dir, band='high_gamma', notch=True, CAR=True,
                      car_chans='average', log_transform=True, do_zscore=True,
                      hg_fs=100, notch_freqs=[60,120,180], overwrite=False,
                      ch_types='ecog', save=False)


# Plotting evoked data

Now that we have some preprocessed data, let's plot the differences between experimental conditions. To do this, we will need the events timings, which are included in the `events.tsv` file. In this case, the events correspond to blocks of music and speech.

## Loading events

Now we will load events from the .tsv file to plot evoked responses to music and speech events. First I'll show you how to do this by creating an MNE events array, next I'll show you how to derive them from the annotations. This first method could also be used with non-BIDS datasets if you have the onset and duration and trial information.

In [ ]:
event_file = f'{ieeg_dir}/sub-{subj}_ses-{sess}_task-{task}_run-{run}_events.tsv'
event_df = pd.read_csv(event_file, delimiter='\t')

In [ ]:
event_df

## Convert event times to samples

Now these event times are in seconds, not samples, so we have to convert them for use with MNE python's epochs constructor. Let's do that here. 

In [ ]:
onset_samp = [int(onset*hgdat.info['sfreq']) for onset in event_df.onset]
dur_samp = [int(dur*hgdat.info['sfreq']) for dur in event_df.duration]
ev_id = [int(e*hgdat.info['sfreq']) for e in event_df.value]

eve = list(zip(onset_samp, dur_samp, ev_id))
print(eve)

## Another way...

So actually, because we already had these particular events as annotations, we could have also done this a simpler way, but the method above also works for other events that are stored in tsv files without becoming annotations.

In [ ]:
events = mne.events_from_annotations(hgdat, event_id='auto')

In [ ]:
events

## Create an epochs object

Now if we want to plot our data by epoch type, we can use the mne Epochs class. This allows us to parse our data according to these events and plot evoked activity.

In [ ]:
tmin = -0.2  # How much time to account for before the event of interest
tmax = 0.5   # How much time to account for after the event of interest
event_id = [3]  # This is the speech event ID
epochs = mne.Epochs(hgdat, events=events[0], tmin=tmin, tmax=tmax, event_id=event_id) 

In [ ]:
# Here we will just plot the average
epochs.plot_image(combine='mean')

In [ ]:
# What about plotting a particular electrode?
epochs.plot_image(picks=[hgdat.info['ch_names'][13]])

In [ ]:
def plot_epochs(epochs, nchans, ch_names, color='b', label='spkr', show=True, vmin_max=None):
    '''
    Function that plots the averaged epoched data for each channel as a grid so you can 
    see all channels at once.
    
    Inputs:
        epochs [obj] : MNE epochs object
        nchans [int] : number of channels to plot
        ch_names [list] : channel names 
        color [str, hex, tuple]: color for ERP traces
        label [str] : label for the ERP (could be epoch type/annotation type) 
        show [bool] : whether to show the figure or not
        vmin_max [list] : list of ylim min and max, e.g. [-0.5, 0.5]
        
    '''
    eps = epochs.get_data()
    emax = np.abs(epochs.average().data).max()
    nrows = int(np.floor(np.sqrt(nchans)))
    ncols = int(np.ceil(nchans/nrows))
    for ch in np.arange(nchans):
        plt.subplot(nrows, ncols, ch+1)
        erp = eps[:,ch,:].mean(0)
        erpstd = eps[:,ch,:].std(0)/np.sqrt(eps.shape[0])
        ybottom = erp - erpstd
        ytop = erp + erpstd

        plt.fill_between(epochs.times, ybottom.ravel(), ytop.ravel(),
                         alpha=0.5, color=color)
        plt.plot(epochs.times, erp, color=color, label=label)
        plt.axvline([0], color='k', linewidth=0.5)
        plt.axhline([0], color='k', linewidth=0.5)
        if vmin_max is None:
            plt.gca().set_ylim([-emax*1.5, emax*1.5])
        else:
            plt.gca().set_ylim([vmin_max[0], vmin_max[1]])
        if ch != 0:
            plt.gca().set_xticks([])
            plt.gca().set_yticks([])
        else:
            plt.ylabel('Z-score')

        plt.text(0.5, 0.25, ch_names[ch], 
            horizontalalignment='center', verticalalignment='center',
            transform=plt.gca().transAxes, fontsize=8)
    plt.gca().set_xticks([epochs.tmin, 0, epochs.tmax])
    plt.xlabel('Time (s)')
    plt.legend()
    #plt.tight_layout()
    if show:
        plt.show()

In [ ]:
plot_epochs(epochs, len(hgdat.info['ch_names']), hgdat.info['ch_names'], label='speech')

## Using stimulus annotations

In addition to the "speech" vs "music" gross-level annotations, the researchers have provided information about the onset and offset of different types of information in the sound as well as the video. You can look in the `stimuli` folder to see what types of annotations are provided, but in general, these include word-level, syllable-level, sentence-level, and specific talkers as well as some other information.

In [ ]:
# Get the word times
annotation = 'words'  # try others here! 
word_times = pd.read_csv(f'{parent_dir}/stimuli/annotations/sound/sound_annotation_{annotation}.tsv', delimiter='\t')
word_times

In [ ]:
start_sample = events[0][events[0][:,2] == events[1]['start task'],0][0]

### Create the new word events epochs

In [ ]:
word_events = []
for idx, row in word_times.iterrows():
    onset_sample = int(row['onset']*hgdat.info['sfreq'])  # convert time to samples
    offset_sample = int(row['offset']*hgdat.info['sfreq'])  # convert time to samples
    duration_sample = offset_sample - onset_sample
    onset_sample+= start_sample  # need to shift by the actual starting time of the task
    
    word_events.append([onset_sample, duration_sample, 1])


In [ ]:
epochs_words = mne.Epochs(hgdat, events=word_events, tmin=-0.2, tmax=1.0)

### Plot word epochs!

In [ ]:
epochs_words.plot_image(combine='mean')

In [ ]:
plot_epochs(epochs_words, len(hgdat.info['ch_names']), hgdat.info['ch_names'], label='words')

In [ ]:
epochs_words.plot_image(picks=['P18'])

### Export data to numpy array

If we want to export the data to use with our own functions, we can also do that with the `.get_data()` method.

In [ ]:
epochs_array = epochs_words.get_data()
print(f'{epochs_array.shape[0]} {annotation} events for\
  {epochs_array.shape[1]} channels and {epochs_array.shape[2]} time points')

In [ ]:
# Use matplotlib to show the average across all epochs
plt.imshow(epochs_array.mean(0))
plt.xlabel('Time')
plt.ylabel('Channel')
plt.gca().set_xticks([0, 
                      -int(epochs_words.tmin*epochs_words.info['sfreq']), 
                      int((epochs_words.tmax - epochs_words.tmin)*epochs_words.info['sfreq'])])
plt.gca().set_xticklabels([epochs_words.tmin, 0, epochs_words.tmax])
plt.axvline(-int(epochs_words.tmin*epochs_words.info['sfreq']), color='w', linestyle='--')

In [ ]:
epochs_words.info['sfreq']

# That's it!

Some suggestions for things you can try:

* Create epochs for different types of events - speech, music, syllables, sentences, etc
* Compare amplitude of speech versus music responses in each electrode
* Look at effects of referencing on the evoked data/epochs.